In [1]:
from load_model import load_model
from accelerate import Accelerator
from dataset import get_dataloader
from tqdm.auto import tqdm
import torch


In [2]:
model_name_or_path = "google/gemma-2b-it"
dataset_path = "../data/daily_dialog_validation.jsonl"
batch_size = 2

model, tokenizer = load_model(
        inference=True, model_weights_name_or_path=model_name_or_path, quantization=4
    )

test_dataloader = get_dataloader(
        dataset_path=dataset_path,
        tokenizer=tokenizer,
        batch_size=batch_size,
    )


/home/iker/miniconda3/envs/transformers/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading dataset from ../data/daily_dialog_validation.jsonl
Loaded 1000 lines from ../data/daily_dialog_validation.jsonl


Preparing input:   0%|          | 0/1000 [00:00<?, ?it/s]

Prepared 4000 examples


In [3]:
accelerator = Accelerator()
model, test_dataloader = accelerator.prepare(model, test_dataloader)

model.eval()

if accelerator.is_local_main_process:
    print("***** Running Evaluation *****")
    print(f"  Num examples = {len(test_dataloader.dataset)}")
    print(f"  Batch size = {batch_size}")

***** Running Evaluation *****
  Num examples = %d 4000
  Batch size = %d 2


In [ ]:
losses = []
for batch in tqdm(test_dataloader, desc="Evaluating"):
    labels = batch.pop("labels")
    loss_weight_mask = batch.pop("loss_weight_mask")
    outputs = model(
        input_ids=batch.input_ids,
        attention_mask=batch.attention_mask,
    )
    logits = outputs["logits"] if isinstance(outputs, dict) else outputs[0]
    break
    logits = logits[..., :-1, :].contiguous()
    labels = labels[..., 1:].contiguous()
    loss_weight_mask = loss_weight_mask[..., 1:].contiguous()

    
    logits = logits.view(-1, logits.size(-1))
    labels = labels.view(-1)
    loss_weight_mask = loss_weight_mask.view(-1)
    